In [8]:
#for plan and executor
%pip install langchain_experimental
# for serpAPI
%pip install google-search-results 
# for LLMMathChain
%pip install langgraph numexpr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_openai import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool

In [58]:
# migration:https://python.langchain.com/api_reference/langchain/chains/langchain.chains.llm_math.base.LLMMathChain.html#llmmathchain
# from langchain.chains.llm_math import LLMMathChain

from typing import Annotated, Sequence

from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from typing import TypedDict

@tool
def calculator(expression: str) -> dict:
    """Calculate mathematical expressions."""
    try:
        import numexpr
        result = float(numexpr.evaluate(expression.strip()))
        return {
            "messages": [{
                "role": "assistant",
                "content": str(result)
            }]
        }
    except Exception as e:
        return {
            "messages": [{
                "role": "assistant",
                "content": f"계산 중 오류가 발생했습니다: {str(e)}"
            }]
        }

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [calculator]
llm_with_tools = llm.bind_tools(tools, tool_choice="any")

class ChainState(TypedDict):
    """LangGraph state."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

def call_model(state: ChainState, config: RunnableConfig) -> dict:
    response = llm.invoke(state["messages"], config)
    return {
        "messages": [{
            "content": response.content if hasattr(response, 'content') else str(response)
        }]
    }

graph_builder = StateGraph(ChainState)
graph_builder.add_node("calculator", calculator)
graph_builder.add_node("call_model", call_model)


graph_builder.set_entry_point("calculator")
graph_builder.add_edge("calculator", "call_model")
graph_builder.add_edge("call_model", END)

llm_math_chain = graph_builder.compile()

# define tools

In [57]:
# SearpAPI needs API key, you can sign in the site and api key https://serpapi.com/users/welcome
search = SerpAPIWrapper(serpapi_api_key="{YOUR_SERPAPI_API_KEY}")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key="{YOUR_OPENAI_KEY}")
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=calculator,
        description="useful for when you need to perform mathematical calculations"
    )
]

# llm = ChatOpenAI(model="gpt-4o", api_key="{YOUR_OPENAI_KEY}", temperature=0)
planner = load_chat_planner(llm)
executor = load_agent_executor(
    llm=llm,
    tools=tools,
    verbose=True
)

agent = PlanAndExecute(
    planner=planner,
    executor=executor,
    verbose=True
)
agent.invoke(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new PlanAndExecute chain...
steps=[Step(value="Identify Leo DiCaprio's current girlfriend as of the latest available information."), Step(value="Determine the current age of Leo DiCaprio's girlfriend."), Step(value='Calculate her age raised to the power of 0.43.'), Step(value="Given the above steps taken, please respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Thought: I need to find the most recent information about Leonardo DiCaprio's current girlfriend. This requires checking the latest news or updates.
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current girlfriend October 2023"
}
```
Observation: Vittoria Ceretti
Thought:Leonardo DiCaprio's current girlfriend as of the latest available information is Vittoria Ceretti. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti."
}
```

> Finished chain.
*****

Step: Identify Leo DiCaprio's cu

{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, who is 25 years old. Her age raised to the power of 0.43 is approximately 3.99."}